# Layers

[![open_in_colab][colab_badge]][colab_notebook_link]
<!-- [![open_in_binder][binder_badge]][binder_notebook_link] -->

[colab_badge]: https://colab.research.google.com/assets/colab-badge.svg
[colab_notebook_link]: https://colab.research.google.com/github/foursquare/fsq-studio-sdk-examples/blob/master/python-notebooks/03%20-%20Layers.ipynb
<!-- [binder_badge]: https://mybinder.org/badge_logo.svg
[binder_notebook_link]: https://mybinder.org/v2/gh/foursquare/fsq-studio-sdk-examples/master?urlpath=lab/tree/python-notebooks/03%20-%20Layers.ipynb -->

This example demonstrates how [map layers](https://docs.unfolded.ai/studio/layer-guide) can be added and controlled in Studio maps.

## Dependencies

This notebook requires the following Python dependencies:

- `foursquare.map-sdk`: The Studio Map SDK
- `pandas`: DataFrame library

If running this notebook in Binder, these dependencies should already be installed. If running in Colab, the next cell will install these dependencies.

In [ ]:
# If in Colab, install this notebook's required dependencies
import sys
if "google.colab" in sys.modules:
    !pip install 'foursquare.map_sdk>=3.0.1' pandas

## Imports

In [2]:
import foursquare.map_sdk as map_sdk
import pandas as pd
from uuid import uuid4

## Using Map Layers

First, let's create a local map:

In [ ]:
map = map_sdk.create_map(
  api_key="<your-api-key>"
)
map

Let's now load the earthquakes dataset so that we can add it to the map:

In [ ]:
url = 'https://4sq-studio-public.s3.us-west-2.amazonaws.com/sdk/examples/sample-data/earthquakes.csv'
earthquakes_data = pd.read_csv(url)

In [ ]:
dataset_id=str(uuid4())

Now we add it and set `auto_create_layers` to `False`, so that we can then manually add a layer for this data:

In [ ]:
earthquakes_data

In [ ]:
map.add_dataset(map_sdk.LocalDatasetCreationProps(
        id=dataset_id,
        label='Earthquakes',
        data=earthquakes_data
    ),
    auto_create_layers = False
)

## Adding a custom layer

Here we create a custom point layer and specify which fields will be used for `lat`, `lon` and color of the points:

In [ ]:
layer = map_sdk.PointLayer(
  id='earthquake_points',
  data_id=dataset_id,
  columns=map_sdk.PointLayerNeighborsColumns(
    lat='Latitude',
    lng='Longitude'
  ),
  label='Earthquakes',
  color_column='Depth',
  color_column_type='real'
)

map.add_layer_from_config(layer.to_json())

## Querying map layers

We can obtain the list of layers which are currently in the map:

In [ ]:
layers = map.get_layers()

`layers` is now a list of layers that exist on the map shown above.

In [ ]:
layers

## Toggle layer visibility

We can also turn the visibility of layers on and off:

In [ ]:
# Hide layer
map.update_layer(
  layer_id=layers[0].id,
  values=map_sdk.LayerUpdateProps(
    is_visible=False
  )
)

In [ ]:
# Show layer again
map.update_layer(
  layer_id=layers[0].id,
  values=map_sdk.LayerUpdateProps(
    is_visible=True
  )
)

## Removing layers

To remove the points layer, which we previously added, run:

In [ ]:
map.remove_layer('earthquake_points')

The following will remove the dataset along with all of its layers:

In [ ]:
map.remove_dataset(dataset_id)